In [1]:
# --- 1. SETUP AND IMPORTS ---
# Import custom modules and necessary third-party libraries.
# `appgeopy` and `my_packages` appear to be user-defined modules.
# It's assumed they contain helper functions and libraries like geopandas, pandas, os, glob, and matplotlib.
from appgeopy import *
from my_packages import *
from sklearn.metrics import (
    mean_absolute_error,
    r2_score,
    root_mean_squared_error,
)

In [2]:
# --- 2. DATA LOADING AND PREPARATION ---

# Load a shapefile containing geospatial point data (monitoring stations) into a GeoDataFrame.
# The 'r' prefix indicates a raw string, which prevents backslashes from being treated as escape characters.
mlcw_gdf = gpd.read_file(
    r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\2_KrigingInterpolation\points_fld\mlcw_twd97.shp"
)
# Display the first 5 rows of the GeoDataFrame to verify it loaded correctly.
mlcw_gdf.head(5)

,STATION,GroundWate,LandSubsid,Y_WGS84,X_WGS84,POINT_X,POINT_Y,PointKey,geometry
0,ANHE,60,MW_AHES,23.52,120.31,179539.204623,2.602035e+06,X179539204Y2602035470,POINT (179539.205 2602035.471)
1,BEICHEN,50,MW_BCES,23.575894,120.303054,178859.958807,2.608229e+06,X178859958Y2608228949,POINT (178859.959 2608228.949)
2,CANLIN,50,MW_TLES,23.574983,120.246516,173088.151033,2.608157e+06,X173088151Y2608157276,POINT (173088.151 2608157.277)
3,DONGGUANG,50,MW_DGES,23.652743,120.272488,175783.144962,2.616755e+06,X175783144Y2616755313,POINT (175783.145 2616755.314)
4,ERLUN,50,MW_ELES,23.771726,120.415522,190429.148778,2.629865e+06,X190429148Y2629865287,POINT (190429.149 2629865.287)


In [3]:
# Define the folder path where model prediction results are stored.
# model_folder = r"D:\1000_SCRIPTS\003_Project002\20250917_GTWR002\4_MainRun"
model_folder = os.getcwd()

# Define the specific kernel name used in the model to find relevant result files.
kernel_name = "tricube"
# Use `glob` to find all files within `model_folder` that contain the kernel name in their filename.
output_files = glob(os.path.join(model_folder, "*", f"*{kernel_name}*.csv"))
# Display the list of found files.
output_files

['D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_1\\gtwr_Layer_1_regression_kernel-tricube_lambda-0d03_bw-25_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_2\\gtwr_Layer_2_regression_kernel-tricube_lambda-0d002_bw-18_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_3\\gtwr_Layer_3_regression_kernel-tricube_lambda-0d07_bw-17_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_4\\gtwr_Layer_4_regression_kernel-tricube_lambda-0d1_bw-18_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_All\\gtwr_Layer_All_regression_kernel-tricube_lambda-0d002_bw-17_results.csv']

In [7]:
# ==============================================================================
# --- 1. SETUP FILE PATHS AND MAIN LOOP ---
# This section is preserved exactly from your original script.
# ==============================================================================

initial_timepoint = datetime(2016, 5, 1)

cache = {
    "STATION": [],
    "Layer": [],
    "r_sq": [],
    "RMSE": [],
    "MAE": [],
    "PBIAS": [],
    "Coeff_Mean": [],
    "Coeff_Stdev": [],
    "Intercept_Mean": [],
    "Intercept_Stdev": [],
}

# Loop through each model output file provided in the 'output_files' list.
for select_file in tqdm(output_files[:], desc="Layer"):
    # Define a general folder for saving the output figures.
    savefig_folder = os.path.join(
        model_folder,
        "y_yhat_figs",
        os.path.dirname(select_file).split("\\")[-1],
    )

    # Create the directory if it doesn't already exist.
    if not os.path.exists(savefig_folder):
        os.makedirs(savefig_folder)

    # Extract a "layer number" from the filename for use in titles and filenames.
    layer_number = os.path.basename(select_file).split("_")[2]

    # --- 2. DATA PROCESSING FOR THE CURRENT FILE ---
    # This logic for creating a PointKey and iterating through points is preserved.

    # Load the selected CSV file into a pandas DataFrame.
    df = pd.read_csv(select_file)

    # # Create a unique identifier ("PointKey") for each data point by combining
    # # its X and Y coordinates. This provides a simple key for grouping by location.
    # pointkey_arr = [
    #     f"X{int(x*1000)}Y{int(y*1000)}"
    #     for x, y in zip(df["X_TWD97"], df["Y_TWD97"])
    # ]
    # df.insert(loc=0, column="PointKey", value=pointkey_arr)
    # df = df.set_index("PointKey")

    # - - - MODIFY THIS LINE TO CONTROL TIME - - -
    df = df.query("Time_value<=68")

    # Get a list of all unique locations in the current file.
    unique_stations = df["input_STATION"].unique().tolist()

    df = df.set_index("input_STATION")

    # Loop through each unique location in the file to generate a plot.
    for select_station in tqdm(unique_stations[:], desc="Station", leave=False):

        try:

            # # Find the station name corresponding to the current PointKey.
            # mlcw_station = mlcw_gdf.query(
            #     "STATION==@select_pointkey"
            # ).STATION.values[0]

            # Filter the DataFrame to get all time-series data for the current location.
            df_byPointKey = df.loc[select_station].copy()

            ################################################################
            # change this line to extract calibration time only
            # df_byPointKey = df_byPointKey.query("input_time<'2021-12-01'")

            # Convert time stamp to datetime objects for proper plotting.
            df_byPointKey["Time_value"] = pd.to_datetime(
                [
                    initial_timepoint + relativedelta(months=time_step)
                    for time_step in df_byPointKey["Time_value"]
                ]
            )

            # --- 3. PREPARE DATA FOR PLOTTING ---

            # Extract data columns into variables for clarity.
            time_arr = df_byPointKey["Time_value"]
            obs_val = df_byPointKey[f"input_Layer_{layer_number}"]
            predict_val = df_byPointKey["predicted_value"]
            # time_arr = df_byPointKey["time_stamp"]
            # obs_val = df_byPointKey["y"]
            # predict_val = df_byPointKey["yhat"]

            # Calculate y-axis limits, adding 20% padding for better visualization.
            # y_range = obs_val.max() - obs_val.min()
            # obs_toplim = obs_val.max() + y_range * 0.2
            # obs_botlim = obs_val.min() - y_range * 0.2
            _top_temp = max(obs_val.max(), predict_val.max())
            _bot_temp = min(obs_val.min(), predict_val.min())
            top_bot_range = abs(_top_temp - _bot_temp)
            obs_toplim = _top_temp + top_bot_range * 0.2
            obs_botlim = _bot_temp - top_bot_range * 0.2

            # --- 4. CALCULATE EVALUATION METRICS ---
            # Calculate key performance indicators to quantify model accuracy.
            r2 = r2_score(obs_val, predict_val)
            rmse = root_mean_squared_error(obs_val, predict_val)
            mae = mean_absolute_error(obs_val, predict_val)
            pbias = 100.0 * np.sum(predict_val - obs_val) / np.sum(obs_val)

            # ==========================================================================
            # --- 5. VISUALIZATION (ENHANCED) ---
            # ==========================================================================

            # Create a figure with 3 subplots stacked vertically.
            fig, axes = plt.subplots(3, 1, figsize=(11.69, 8.27), sharex=True)

            # Set the main title, incorporating the station and layer number.
            fig.suptitle(
                f"{select_station} - Layer {layer_number}",
                y=0.975,
                fontweight="bold",
                fontsize=20,
            )

            # --- Plot 1: Observations vs. Predictions ---
            ax1 = axes[0]
            ax1.plot(
                time_arr,
                obs_val,
                color="darkgrey",
                ls="--",
                linewidth=2,
                marker="o",
                ms=8,
                label="Obs",
                markevery=3,
            )
            ax1.plot(
                time_arr,
                predict_val,
                color="dodgerblue",
                linestyle=(0, (1, 1)),
                marker="s",
                ms=8,
                linewidth=2,
                label="Pred",
                markevery=3,
            )
            ax1.set_ylim(bottom=obs_botlim, top=obs_toplim)

            # Add a text box with performance metrics.
            stats_text = (
                f"R²   : {r2:.3f}\n"
                f"RMSE : {rmse:.3f}\n"
                f"MAE  : {mae:.3f}\n"
                f"PBIAS: {pbias:.2f}%"
            )
            ax1.text(
                0.98,
                0.97,
                stats_text,
                transform=ax1.transAxes,
                ha="right",
                va="top",
                fontsize=10,
                fontfamily="monospace",
                bbox=dict(
                    boxstyle="round,pad=0.5", fc="#EAEAF2", ec="black", lw=1
                ),
            )

            # --- Plot 2: GTWR Coefficients ---
            ax2 = axes[1]

            ax2.plot(
                time_arr,
                df_byPointKey["CUMDISP"],  # Assumed to be a model coefficient
                color="darkviolet",
                # linestyle="--",  # Use a dashed line
                marker="s",  # Use square markers
                markerfacecolor="none",
                ms=8,
                linewidth=2,
                markevery=3,
            )
            ax2.set_ylabel("GTWR\nCoefficients", fontweight="bold")

            # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
            coeff_mean = np.mean(df_byPointKey["CUMDISP"])
            coeff_stdev = np.std(df_byPointKey["CUMDISP"])

            ax2.axhline(coeff_mean, lw=1, color="darkmagenta", ls="--")

            ax2.text(
                x=0.025,
                y=0.1,
                s=r"$\overline{\beta_1}$="
                + f"{coeff_mean:.2f}±{coeff_stdev:.2f}",
                transform=ax2.transAxes,
                fontsize=14,
            )

            # --- Plot 3: Model Intercept ---
            ax3 = axes[2]
            ax3.plot(
                time_arr,
                df_byPointKey["Intercept"],
                color="darkgreen",
                marker="D",
                markerfacecolor="none",
                ms=8,
                linewidth=2,
                markevery=3,
            )

            ax3.axhline(
                np.mean(df_byPointKey["Intercept"]),
                lw=1,
                color="green",
                ls="--",
            )

            intercept_mean = np.mean(df_byPointKey["Intercept"])
            intercept_stdev = np.std(df_byPointKey["Intercept"])

            ax3.text(
                x=0.025,
                y=0.1,
                s=r"$\overline{\beta_0}$="
                + f"{intercept_mean:.2f}±{intercept_stdev:.2f}",
                transform=ax3.transAxes,
                fontsize=14,
            )

            # --- 6. FINAL STYLING AND OUTPUT ---

            for ax in axes:
                # `visualize.configure_axis` is a custom function to style the axis ticks and borders.
                visualize.configure_axis(
                    ax=ax,
                    tick_direction="out",
                    hide_spines=["top", "right"],
                    major_tick_length=10,
                    minor_tick_length=7,
                )

                visualize.configure_datetime_ticks(
                    ax=ax,
                    major_interval=12,
                    minor_interval=3,
                    fontsize=14,
                    grid=False,
                    start_date=datetime(2016, 1, 1),
                    end_date=datetime(
                        2022, 1, 1
                    ),  # modify this control shown time range
                )

            # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
            # Force first tick at 2016-1-1
            for ax in axes:
                current_ticks = list(ax.get_xticks())
                first_tick = mdates.date2num(datetime(2016, 1, 1))
                ax.set_xticks([first_tick] + current_ticks)
            # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
            # Generate dates every 6 months from start to end
            start = datetime(2016, 1, 1)
            end = datetime(2022, 4, 1)
            current = start
            vline_dates = []

            while current <= end:
                vline_dates.append(current)
                current += relativedelta(months=6)

            # Draw vertical lines on all subplots
            for ax in axes:
                for vline_date in vline_dates:
                    ax.axvline(
                        x=vline_date,
                        color="gray",
                        linestyle="--",
                        linewidth=1,
                        alpha=0.5,
                        zorder=0,  # Draw behind other elements
                    )

            # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
            # `visualize.configure_legend` is another custom function for legend styling.
            visualize.configure_legend(
                ax=axes[0],
                columnspacing=0.5,
                labelspacing=0.1,
                handletextpad=0.2,
                ncol=2,
                fontsize_base=14,
                loc="lower left",
            )

            ax1.set_ylabel(
                "Cumulative\nCompaction (mm)",
                fontsize=14,
                loc="center",
                fontweight="bold",
            )
            ax2.set_ylabel(
                "GTWR\nCoefficient",
                fontsize=14,
                loc="center",
                fontweight="bold",
            )
            ax3.set_ylabel(
                "GTWR\nIntercept", fontsize=14, loc="center", fontweight="bold"
            )

            # Adjust layout to prevent titles and labels from overlapping.
            fig.tight_layout(rect=[0, 0.05, 1, 0.96])
            fig.autofmt_xdate(ha="center", rotation=90)

            for key, value in zip(
                cache.keys(),
                [
                    select_station,
                    layer_number,
                    r2,
                    rmse,
                    mae,
                    pbias,
                    coeff_mean,
                    coeff_stdev,
                    intercept_mean,
                    intercept_stdev,
                ],
            ):
                cache[key].append(value)

            # Save the figure using your original filename format.
            save_filename = (
                "_".join([select_station, "layer", layer_number]) + ".png"
            )

            visualize.save_figure(
                fig=fig,
                savepath=os.path.join(savefig_folder, save_filename),
            )

            # Close the figure to free up memory.
            plt.close(fig)
        except Exception as e:
            print(select_pointkey, e)
            pass

    summary_table = pd.DataFrame(cache)
    summary_table.to_excel(
        os.path.join(
            savefig_folder, f"summary_table_Layer_{layer_number}.xlsx"
        ),
        index=False,
    )

Layer:   0%|          | 0/5 [00:00<?, ?it/s]

Station:   0%|          | 0/29 [00:00<?, ?it/s]

Station:   0%|          | 0/29 [00:00<?, ?it/s]

Station:   0%|          | 0/28 [00:00<?, ?it/s]

Station:   0%|          | 0/25 [00:00<?, ?it/s]

Station:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
savefig_folder

In [ ]:
show(mlcw_gdf)